Cargar los archivos CSV

In [50]:
import pandas as pd

# Cargar los archivos CSV
calificaciones_df = pd.read_csv('input/calificaciones.csv')
carrera_df = pd.read_csv('input/carrera.csv')
escuelas_df = pd.read_csv('input/escuelas.csv')
estudiantes_df = pd.read_csv('input/estudiantes.csv')

# Ver las primeras filas de cada DataFrame
print(calificaciones_df.head())
print(carrera_df.head())
print(escuelas_df.head())
print(estudiantes_df.head())

   periodo  clave_alumno clave_materia  calificacion  tipo_calificacion
0     2001             0       ACB9309         100.0                  1
1     2001             0       ACH9322          75.0                  1
2     2001             0       ACM9313           0.0                  3
3     2001             0       ACM9301           0.0                  3
4     2001             0       ACC9326          93.0                  1
   clave_carrera                  nombre        corto
0              0   INGENIERIA INDUSTRIAL   Industrial
1              1   INGENIERIA BIOQUIMICA   Bioquimica
2              2     INGENIERIA MECANICA     Mecanica
3              3    INGENIERIA ELECTRICA    Electrica
4              4  INGENIERIA ELECTRONICA  Electronica
   clave                         escuela
0      3  COBAES 3(PROFRA. VELINA LEON )
1      4  COBAES 4(PROFR. VICTOR M. RUB)
2      5  COBAES 5(PROFRA. FCA. GUERRER)
3      6  COBAES 6(ING.JOSE HERNANDEZ T)
4      7  COBAES 7(GRAÑ. PABLO MACIAS V

Explorar los datos
Verificamos si los datos contienen valores nulos, duplicados, o alguna inconsistencia:

In [51]:
# Verificar si hay valores nulos
print("Valores nulos en calificaciones:\n", calificaciones_df.isnull().sum())
print("Valores nulos en carrera:\n", carrera_df.isnull().sum())
print("Valores nulos en escuelas:\n", escuelas_df.isnull().sum())
print("Valores nulos en estudiantes:\n", estudiantes_df.isnull().sum())

# Verificar si hay duplicados
print("Duplicados en calificaciones:", calificaciones_df.duplicated().sum())
print("Duplicados en carrera:", carrera_df.duplicated().sum())
print("Duplicados en escuelas:", escuelas_df.duplicated().sum())
print("Duplicados en estudiantes:", estudiantes_df.duplicated().sum())

Valores nulos en calificaciones:
 periodo              0
clave_alumno         0
clave_materia        0
calificacion         0
tipo_calificacion    0
dtype: int64
Valores nulos en carrera:
 clave_carrera    0
nombre           0
corto            0
dtype: int64
Valores nulos en escuelas:
 clave      0
escuela    0
dtype: int64
Valores nulos en estudiantes:
 clave_alumno        0
clave_carrera       0
sexo                0
prepa               0
colonia           600
municipio         363
ciudad            287
ingreso             1
termino          5317
causa_salida        0
dtype: int64
Duplicados en calificaciones: 4
Duplicados en carrera: 0
Duplicados en escuelas: 0
Duplicados en estudiantes: 0


Limpieza de los datos
Dependiendo de lo que encuentres en la exploración, puedes hacer lo siguiente:

Eliminar duplicados:

In [52]:
# Eliminar duplicados
calificaciones_df = calificaciones_df.drop_duplicates()
carrera_df = carrera_df.drop_duplicates()
escuelas_df = escuelas_df.drop_duplicates()
estudiantes_df = estudiantes_df.drop_duplicates()

Manejo de valores nulos:

Dependiendo de los valores nulos, puedes optar por eliminar filas o rellenarlas:

In [53]:
# Rellenar valores nulos con un valor específico (ejemplo: 0 o 'Desconocido')
calificaciones_df = calificaciones_df.fillna({'calificacion': 0, 'tipo_calificacion': 'Desconocido'})
carrera_df = carrera_df.fillna({'nombre': 'Desconocido', 'corto': 'Desconocido'})
escuelas_df = escuelas_df.fillna({'escuela': 'Desconocida'})
estudiantes_df = estudiantes_df.fillna({'prepa': 'Desconocida', 'colonia': 'Desconocida', 'municipio': 'Desconocido'})

# O eliminar filas con valores nulos si es necesario
# calificaciones_df = calificaciones_df.dropna()


Reemplazar valores incorrectos o inconsistentes:

Si encuentras valores que no son correctos (por ejemplo, valores numéricos negativos donde no deberían ser), puedes reemplazarlos:

In [54]:
# Reemplazar valores negativos por 0 en calificación
calificaciones_df['calificacion'] = calificaciones_df['calificacion'].apply(lambda x: max(x, 0))


Revisar tipos de datos
Es importante verificar si los tipos de datos son correctos (por ejemplo, fechas como strings, números como strings, etc.).

In [55]:
# Verificar tipos de datos
print(estudiantes_df.dtypes)

# Convertir columnas a tipos correctos si es necesario
estudiantes_df['ingreso'] = pd.to_datetime(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_datetime(estudiantes_df['termino'], errors='coerce')


clave_alumno       int64
clave_carrera      int64
sexo               int64
prepa              int64
colonia           object
municipio         object
ciudad            object
ingreso          float64
termino          float64
causa_salida       int64
dtype: object


Unir los DataFrames
Si tienes que combinar los DataFrames en uno solo para hacer el análisis, usa merge():

In [56]:
# Verificar las columnas de los DataFrames
print(estudiantes_df.columns)
print(escuelas_df.columns)

# Unir estudiantes con carreras
estudiantes_calificaciones_escuelas = pd.merge(estudiantes_df, calificaciones_df, on='clave_alumno', how='left')
estudiantes_carrera = pd.merge(estudiantes_calificaciones_escuelas, carrera_df, on='clave_carrera', how='left')

# Unir con escuelas usando 'clave_carrera' de estudiantes y 'clave' de escuelas
estudiantes_completos = pd.merge(estudiantes_carrera, escuelas_df, left_on='clave_carrera', right_on='clave', how='left')

# Verificar el resultado
print(estudiantes_completos.head())


Index(['clave_alumno', 'clave_carrera', 'sexo', 'prepa', 'colonia',
       'municipio', 'ciudad', 'ingreso', 'termino', 'causa_salida'],
      dtype='object')
Index(['clave', 'escuela'], dtype='object')
   clave_alumno  clave_carrera  sexo  prepa      colonia municipio  \
0            28              4     1      2  Los Angeles   25006.0   
1            28              4     1      2  Los Angeles   25006.0   
2            28              4     1      2  Los Angeles   25006.0   
3            28              4     1      2  Los Angeles   25006.0   
4            28              4     1      2  Los Angeles   25006.0   

           ciudad                       ingreso                       termino  \
0  CULIACAN, SIN. 1970-01-01 00:00:00.000002003 1970-01-01 00:00:00.000002061   
1  CULIACAN, SIN. 1970-01-01 00:00:00.000002003 1970-01-01 00:00:00.000002061   
2  CULIACAN, SIN. 1970-01-01 00:00:00.000002003 1970-01-01 00:00:00.000002061   
3  CULIACAN, SIN. 1970-01-01 00:00:00.000002003 1970

Errores ortográficos en claves y datos categóricos:
- Normalizar cadenas:
- Reemplazar errores comunes:

In [57]:
estudiantes_df['ciudad'] = estudiantes_df['ciudad'].str.lower().str.strip()
escuelas_df['escuela'] = escuelas_df['escuela'].str.lower().str.strip()

estudiantes_df['ciudad'] = estudiantes_df['ciudad'].replace({
    'cdmx': 'ciudad de mexico', 
    'mexico': 'ciudad de mexico'
})


Formato de fechas incorrecto:

In [58]:
estudiantes_df['ingreso'] = pd.to_datetime(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_datetime(estudiantes_df['termino'], errors='coerce')


Números con letras (mezcla de datos numéricos y texto):

In [59]:
estudiantes_df['ingreso'] = pd.to_numeric(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_numeric(estudiantes_df['termino'], errors='coerce')

Detectar filas con datos incorrectos:

In [60]:
# Convertir las columnas a numérico, reemplazando los valores no numéricos por NaN
estudiantes_df['ingreso'] = pd.to_numeric(estudiantes_df['ingreso'], errors='coerce')
estudiantes_df['termino'] = pd.to_numeric(estudiantes_df['termino'], errors='coerce')


Números con letras (mezcla de datos numéricos y texto):

In [61]:

# Filtrar las filas donde 'ingreso' o 'termino' es NaN (lo que indica que eran datos no numéricos)
ingreso_no_numerico = estudiantes_df[estudiantes_df['ingreso'].isna()]
termino_no_numerico = estudiantes_df[estudiantes_df['termino'].isna()]

# Mostrar las filas con datos no numéricos en 'ingreso' o 'termino'
print("Filas con ingreso no numérico:")
print(ingreso_no_numerico)
print("\nFilas con termino no numérico:")
print(termino_no_numerico)


Filas con ingreso no numérico:
Empty DataFrame
Columns: [clave_alumno, clave_carrera, sexo, prepa, colonia, municipio, ciudad, ingreso, termino, causa_salida]
Index: []

Filas con termino no numérico:
Empty DataFrame
Columns: [clave_alumno, clave_carrera, sexo, prepa, colonia, municipio, ciudad, ingreso, termino, causa_salida]
Index: []


Reemplazo de valores nulos después de la limpieza:

In [62]:
estudiantes_completos['ciudad'].fillna('Desconocida', inplace=True)
estudiantes_completos['ingreso'].fillna(0, inplace=True)
estudiantes_completos['termino'].fillna(0, inplace=True)
estudiantes_completos['escuela'].fillna('Desconocida', inplace=True)


/tmp/ipykernel_11444/3445933231.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  estudiantes_completos['ciudad'].fillna('Desconocida', inplace=True)
/tmp/ipykernel_11444/3445933231.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value,

Eliminar filas con valores nulos en columnas clave (calificación, materia, etc.)

In [63]:
# Eliminar filas con valores nulos en las columnas de calificaciones
estudiantes_completos_limpios = estudiantes_completos.dropna(subset=['periodo', 'clave_materia', 'calificacion', 'tipo_calificacion'])

# Guardar el DataFrame limpio
estudiantes_completos_limpios.to_csv('estudiantes_completos_limpios.csv', index=False)


Rellenar valores nulos con un valor predeterminado (por ejemplo, 0 para las calificaciones)

In [64]:
# Rellenar los valores nulos en las columnas clave con 0 o algún valor predeterminado
estudiantes_completos['calificacion'] = estudiantes_completos['calificacion'].fillna(0)
estudiantes_completos['periodo'] = estudiantes_completos['periodo'].fillna('Desconocido')  # O algún valor que se adapte
estudiantes_completos['clave_materia'] = estudiantes_completos['clave_materia'].fillna('Desconocido')
estudiantes_completos['tipo_calificacion'] = estudiantes_completos['tipo_calificacion'].fillna('Desconocido')
estudiantes_completos['clave'] = estudiantes_completos['clave'].fillna('Desconocido')  # Para la escuela

# Guardar el DataFrame limpio
estudiantes_completos.to_csv('estudiantes_completos_limpios.csv', index=False)


Solo verificar y visualizar las filas con valores nulos

In [65]:
# Ver las filas con valores nulos en las columnas clave
nulos = estudiantes_completos[estudiantes_completos.isnull().any(axis=1)]
print(nulos)

Empty DataFrame
Columns: [clave_alumno, clave_carrera, sexo, prepa, colonia, municipio, ciudad, ingreso, termino, causa_salida, periodo, clave_materia, calificacion, tipo_calificacion, nombre, corto, clave, escuela]
Index: []


In [66]:
# Convertir todas las columnas numéricas a tipo float
estudiantes_df['ingreso'] = pd.to_numeric(estudiantes_df['ingreso'], errors='coerce', downcast='float')
estudiantes_df['termino'] = pd.to_numeric(estudiantes_df['termino'], errors='coerce', downcast='float')

# También se puede aplicar a otras columnas numéricas si las hubiera
# Asegurarse de que todas las columnas numéricas sean float
for col in estudiantes_df.select_dtypes(include=['int64', 'float64']).columns:
    estudiantes_df[col] = estudiantes_df[col].astype('float')

# Verificar el tipo de datos después de la conversión
print(estudiantes_df.dtypes)

clave_alumno     float64
clave_carrera    float64
sexo             float64
prepa            float64
colonia           object
municipio         object
ciudad            object
ingreso          float32
termino          float32
causa_salida     float64
dtype: object


Verificación final y exportación
Después de la limpieza, puedes revisar el DataFrame final y guardarlo para su análisis posterior:

In [67]:
# Verificar si hay valores nulos en el DataFrame final
print(estudiantes_completos.isnull().sum())

# Guardar el DataFrame limpio
estudiantes_completos.to_csv('estudiantes_completos_limpios.csv', index=False)

clave_alumno         0
clave_carrera        0
sexo                 0
prepa                0
colonia              0
municipio            0
ciudad               0
ingreso              0
termino              0
causa_salida         0
periodo              0
clave_materia        0
calificacion         0
tipo_calificacion    0
nombre               0
corto                0
clave                0
escuela              0
dtype: int64


In [68]:
# Cargar el archivo CSV para visualizarlo
estudiantes_completos_guardados = pd.read_csv('estudiantes_completos_limpios.csv')

# Ver las primeras filas
print(estudiantes_completos_guardados.head())


   clave_alumno  clave_carrera  sexo  prepa      colonia municipio  \
0            28              4     1      2  Los Angeles   25006.0   
1            28              4     1      2  Los Angeles   25006.0   
2            28              4     1      2  Los Angeles   25006.0   
3            28              4     1      2  Los Angeles   25006.0   
4            28              4     1      2  Los Angeles   25006.0   

           ciudad                        ingreso  \
0  CULIACAN, SIN.  1970-01-01 00:00:00.000002003   
1  CULIACAN, SIN.  1970-01-01 00:00:00.000002003   
2  CULIACAN, SIN.  1970-01-01 00:00:00.000002003   
3  CULIACAN, SIN.  1970-01-01 00:00:00.000002003   
4  CULIACAN, SIN.  1970-01-01 00:00:00.000002003   

                         termino  causa_salida periodo clave_materia  \
0  1970-01-01 00:00:00.000002061             5  2003.0        04tBt2   
1  1970-01-01 00:00:00.000002061             5  2003.0        04tBt1   
2  1970-01-01 00:00:00.000002061             5  20

/tmp/ipykernel_11444/3039516880.py:2: DtypeWarning: Columns (5,10,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  estudiantes_completos_guardados = pd.read_csv('estudiantes_completos_limpios.csv')
